In [22]:
import os, openai, json, importlib
import keys, modify_spec
os.environ["OPENAI_API_KEY"] = keys.OPEN_AI_KEY
os.environ["HELIUS_API_KEY"] = keys.HELIUS_KEY
openai.api_key = keys.OPEN_AI_KEY
from langchain.utilities.openapi import OpenAPISpec

# local version of openapi script from langchain to experiment with
import openapi


In [23]:
# with open("local_solana_openapi.yaml") as f:
#     raw_solana_api_spec = yaml.load(f, Loader=yaml.Loader)
# solana_api_spec = reduce_openapi_spec(raw_solana_api_spec)
# print(len(solana_api_spec.endpoints))
# # pretty print the first endpoint
# print(json.dumps(solana_api_spec.endpoints[0], indent=2))
# # print(type(solana_api_spec.endpoints[0]))

In [24]:
spec = "local_solana_openapi.yaml"
if isinstance(spec, str):
    for conversion in (
        OpenAPISpec.from_url,
        OpenAPISpec.from_file,
        OpenAPISpec.from_text,
    ):
        try:
            spec = conversion(spec)  # type: ignore[arg-type]
            break
        except Exception:  # noqa: E722
            pass
    if isinstance(spec, str):
        raise ValueError(f"Unable to parse spec from source {spec}")
openai_fns, call_api_fn = openapi.openapi_spec_to_openai_fn(spec)


Attempting to load an OpenAPI 3.0.2 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


In [25]:
print(call_api_fn)

<function openapi_spec_to_openai_fn.<locals>.default_call_api at 0x1384236a0>


In [69]:
print(json.dumps(openai_fns[0], indent = 2))

{
  "name": "query_assets_by_owner",
  "description": "Accepts Solana publicKey address. Returns Metaplex NFTs owned by the address",
  "parameters": {
    "type": "object",
    "properties": {
      "json": {
        "properties": {
          "address": {
            "type": "string",
            "title": "Address"
          }
        },
        "type": "object",
        "required": [
          "address"
        ],
        "title": "GetAccountInfoRequest"
      }
    }
  }
}


In [77]:
# to experiment with imported scripts we need to reload them again and again
%load_ext autoreload
%autoreload 2
# could use the below command to reload just one script
# importlib.reload(openapi)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
chain = openapi.get_openapi_chain("local_solana_openapi.yaml", verbose = True)
# don't print chain otherwise api-key will be printed

Attempting to load an OpenAPI 3.0.2 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


{'name': 'query_assets_by_owner', 'description': 'Accepts Solana publicKey address. Returns Metaplex NFTs owned by the address', 'parameters': {'type': 'object', 'properties': {'json': {'properties': {'address': {'type': 'string', 'title': 'Address'}}, 'type': 'object', 'required': ['address'], 'title': 'GetAccountInfoRequest'}}}}
{'name': 'query_account_info', 'description': 'Accepts Solana publicKey address. Returns information the account data', 'parameters': {'type': 'object', 'properties': {'json': {'properties': {'address': {'type': 'string', 'title': 'Address'}}, 'type': 'object', 'required': ['address'], 'title': 'GetAccountInfoRequest'}}}}
{'name': 'query_token_accounts', 'description': 'Returns the fungible and non-fungible tokens and amounts owned by the address. May show tokens not listed in /getAssetsByOwner', 'parameters': {'type': 'object', 'properties': {'json': {'properties': {'address': {'type': 'string', 'title': 'Address'}}, 'type': 'object', 'required': ['address']

### Below code works alright

In [58]:
# getBalance endpoint
chain.run("Could you tell me the balance in the account by the address 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625 ?")



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Human: Use the provided API's to respond to this user query:

Could you tell me the balance in the account by the address 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625 ?

> Finished chain.


> Entering new  chain...
Calling endpoint query_balance with arguments:
{
  "json": {
    "address": "8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625"
  }
}post http://localhost:3000/api/handlers/getBalance {'json': {'address': '8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625'}}

> Finished chain.

> Finished chain.


{'sol': 0.042545265}

In [59]:
# getAccountInfo endpoint
chain.run("Could you tell me information about the account at address 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625 ?")



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Human: Use the provided API's to respond to this user query:

Could you tell me information about the account at address 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625 ?

> Finished chain.


> Entering new  chain...
Calling endpoint query_account_info with arguments:
{
  "json": {
    "address": "8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625"
  }
}post http://localhost:3000/api/handlers/getAccountInfo {'json': {'address': '8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625'}}

> Finished chain.

> Finished chain.


{'message': '{"data":{"type":"Buffer","data":[]},"executable":false,"lamports":42545265,"owner":"11111111111111111111111111111111","rentEpoch":0}'}

In [60]:
# getTokenAccounts endpoint
chain.run("Could you tell all the tokens owned by address 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625 ?")



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Human: Use the provided API's to respond to this user query:

Could you tell all the tokens owned by address 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625 ?

> Finished chain.


> Entering new  chain...
Calling endpoint query_token_accounts with arguments:
{
  "json": {
    "address": "8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625"
  }
}post http://localhost:3000/api/handlers/getTokenAccounts {'json': {'address': '8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625'}}

> Finished chain.

> Finished chain.


[{'mint': 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v', 'amount': '95'}]

In [72]:
# getAssetsByOwner endpoint
chain.run("Could you tell me all the NFTs owned by account 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625?")
# Locally hosted getAssetsByOwner endpoint has an issue with helius api key, and is not working



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Human: Use the provided API's to respond to this user query:

Could you tell me all the NFTs owned by account 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625?

> Finished chain.


> Entering new  chain...
Calling endpoint query_assets_by_owner with arguments:
{
  "json": {
    "address": "8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625"
  }
}post http://localhost:3000/api/handlers/getAssetsByOwner {'json': {'address': '8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625'}}


KeyError: 'params'

In [65]:
# getTransaction endpoint
chain.run("Could you tell me about the transaction vu7KU1DjPKmuPLWejwER14T7u9ywnH1CjF3WSXCsWXgn7yfbe4JUT61HSa2w2UUc8KcgvkFAbDjnNMGYKQfdiNb ?")



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Human: Use the provided API's to respond to this user query:

Could you tell me about the transaction vu7KU1DjPKmuPLWejwER14T7u9ywnH1CjF3WSXCsWXgn7yfbe4JUT61HSa2w2UUc8KcgvkFAbDjnNMGYKQfdiNb ?

> Finished chain.


> Entering new  chain...
Calling endpoint query_transaction with arguments:
{
  "json": {
    "signature": "vu7KU1DjPKmuPLWejwER14T7u9ywnH1CjF3WSXCsWXgn7yfbe4JUT61HSa2w2UUc8KcgvkFAbDjnNMGYKQfdiNb"
  }
}post http://localhost:3000/api/handlers/getTransaction {'json': {'signature': 'vu7KU1DjPKmuPLWejwER14T7u9ywnH1CjF3WSXCsWXgn7yfbe4JUT61HSa2w2UUc8KcgvkFAbDjnNMGYKQfdiNb'}}

> Finished chain.

> Finished chain.


{'txFeeInLamports': 5000,
 'computeUnitsConsumed': 0,
 'txVersion': '"legacy"',
 'logs': ['Program Vote111111111111111111111111111111111111111 invoke [1]',
  'Program Vote111111111111111111111111111111111111111 success'],
 'feePayer': 'GQzMeEMwAR44ugoNCifTb5NdRKos1GduDUPeNh6AgV46',
 'instructions': [{'programId': 'Vote111111111111111111111111111111111111111',
   'ixData': 'AgAAAAIAAAAAAAAAqFYODAAAAACpVg4MAAAAAMe89CDdwzo3FKA0Y7JfwJZco2K1ScNqn7OCtSMoKgaRAbOinGQAAAAA',
   'keys': ['oixpqSNX7CKWHw93ViA8u1CcLzZXDmacKJjV4AvxMZE',
    'SysvarS1otHashes111111111111111111111111111',
    'SysvarC1ock11111111111111111111111111111111',
    'GQzMeEMwAR44ugoNCifTb5NdRKos1GduDUPeNh6AgV46'],
   'depth': 0}],
 'tokenBalanceChanges': {},
 'solBalanceChanges': {}}

In [81]:
# getSignaturesForAddress endpoint
chain.run("Could you tell me the transaction signatures performed by address oixpqSNX7CKWHw93ViA8u1CcLzZXDmacKJjV4AvxMZE?")



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Human: Use the provided API's to respond to this user query:

Could you tell me the transaction signatures performed by address oixpqSNX7CKWHw93ViA8u1CcLzZXDmacKJjV4AvxMZE?

> Finished chain.


> Entering new  chain...
Calling endpoint query_signatures_for_address with arguments:
{
  "json": {
    "address": "oixpqSNX7CKWHw93ViA8u1CcLzZXDmacKJjV4AvxMZE",
    "beforeSignature": "",
    "untilSignature": ""
  }
}None
post http://localhost:3000/api/handlers/getSignaturesForAddress {'json': {'address': 'oixpqSNX7CKWHw93ViA8u1CcLzZXDmacKJjV4AvxMZE', 'beforeSignature': '', 'untilSignature': ''}}
500


KeyError: 'params'

In [70]:
print(json.dumps(openai_fns[5], indent = 2))

{
  "name": "query_signatures_for_address",
  "description": "Accepts Solana publicKey address. Returns the latest transaction signatures that involve that address",
  "parameters": {
    "type": "object",
    "properties": {
      "json": {
        "properties": {
          "address": {
            "type": "string"
          },
          "beforeSignature": {
            "type": "string"
          },
          "untilSignature": {
            "type": "string"
          }
        },
        "type": "object",
        "required": [
          "address"
        ],
        "title": "GetSignaturesForAddressRequest"
      }
    }
  }
}


### Issue with getSignaturesForAddress endpoint
- The request we are generating has before signature and until signature as empty strings, and this can't be handled by our backend. But when we don't pass them we can still get the right reply. Verified this by manually making requests through postman. 
- Since the required field is only address, why does the generated request have all the params. This is maybe we could improve on the request generation part. 
- On the above point we could look if the beforeSignature and afterSignature is being generated by the API or is it being compulsorily added by the openapi.py script from langchain. If that is the case - remove the case and maybe put a PR on langchain repo. 

### Interesting Conclusions
- When the langchain error is something like KeyError: 'params', it means that the API call was not okay. 

In [74]:
# getCollectionsByFloorPrice endpoint
chain.run("Could you tell me the NFTs available priced between 20 and 30 SOL?")
# Hitting the right endpoint but issue with the hosteed version



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Human: Use the provided API's to respond to this user query:

Could you tell me the NFTs available priced between 20 and 30 SOL?

> Finished chain.


> Entering new  chain...
Calling endpoint query_nft_collections_by_fp with arguments:
{
  "json": {
    "minFloorPrice": 20,
    "maxFloorPrice": 30,
    "orderBy": "ASC"
  }
}post http://localhost:3000/api/handlers/getCollectionsByFloorPrice {'json': {'minFloorPrice': 20, 'maxFloorPrice': 30, 'orderBy': 'ASC'}}


KeyError: 'params'

In [75]:
# getCollectionsByFloorPrice endpoint
chain.run("Could you tell me the NFTs available priced between 20 and 30 SOL?")
# Hitting the right endpoint but issue with the hosteed version



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Human: Use the provided API's to respond to this user query:

Could you tell me the NFTs available priced between 20 and 30 SOL?

> Finished chain.


> Entering new  chain...
Calling endpoint query_nft_collections_by_fp with arguments:
{
  "json": {
    "minFloorPrice": 20,
    "maxFloorPrice": 30,
    "orderBy": "ASC"
  }
}post http://localhost:3000/api/handlers/getCollectionsByFloorPrice {'json': {'minFloorPrice': 20, 'maxFloorPrice': 30, 'orderBy': 'ASC'}}


KeyError: 'params'